In [18]:
import random

In [19]:
def load_edges(path):
    edges = []
    with open(path, "r") as f:
        for line in f:
            edge = line.strip().split()
            if edge:
                edges.append(frozenset(edge))
    return edges

edges = load_edges("soc-Epinions1.txt")
print(f"Loaded {len(edges)} edges.")
edges[:5]

Loaded 508837 edges.


[frozenset({'0', '4'}),
 frozenset({'0', '5'}),
 frozenset({'0', '7'}),
 frozenset({'0', '8'}),
 frozenset({'0', '9'})]

In [104]:
def reservoir_sampling_base(edge, k, reservoir, i):
    if i < k:
        [reservoir.append(edge), True]
    else:
        p = random.randint(0, i)
        if p < k:
            replace = random.randint(0, k - 1)
            reservoir[replace] = edge
            return [reservoir, True]
    return [reservoir, False]

k = 1000
reservoir = []

for (i, edge) in enumerate(edges):
    [reservoir, _] = reservoir_sampling_base(edge, k, reservoir, i)
print(reservoir)
print(len(reservoir))

[frozenset({'4804', '2678'}), frozenset({'19561', '2066'}), frozenset({'29628', '9753'}), frozenset({'5131', '1410'}), frozenset({'3683', '3064'}), frozenset({'18343', '3986'}), frozenset({'8268', '9169'}), frozenset({'9704', '1021'}), frozenset({'1572', '2696'}), frozenset({'381', '934'}), frozenset({'23424', '59650'}), frozenset({'9320', '19313'}), frozenset({'1129', '1108'}), frozenset({'681', '19737'}), frozenset({'3854', '2435'}), frozenset({'2232', '1181'}), frozenset({'8412', '279'}), frozenset({'1037', '2042'}), frozenset({'3278', '130'}), frozenset({'2064', '689'}), frozenset({'1349', '2098'}), frozenset({'1532', '4123'}), frozenset({'21611', '52665'}), frozenset({'849', '223'}), frozenset({'8183', '1847'}), frozenset({'10106', '20569'}), frozenset({'11872', '50064'}), frozenset({'1179', '10950'}), frozenset({'4610', '32627'}), frozenset({'6946', '1489'}), frozenset({'67878', '48633'}), frozenset({'7047', '42897'}), frozenset({'8777', '46104'}), frozenset({'1596', '1178'}), fr

In [105]:
def generate_wedges(edge, edge_reservoir):
    wedges = []
    total_wedges = 0
    for res_edge in edge_reservoir:
        if len(edge.intersection(res_edge)) == 1:
            wedges.append((edge, res_edge))
            total_wedges += 1
    return [total_wedges, wedges]

print(generate_wedges(frozenset(['1', '2']), reservoir))


[3, [(frozenset({'2', '1'}), frozenset({'1', '19290'})), (frozenset({'2', '1'}), frozenset({'421', '1'})), (frozenset({'2', '1'}), frozenset({'8884', '1'}))]]
